In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys, os
sys.path.append('../')
import mg_si
from imp import reload
reload(mg_si)
import pandas as pd
import scipy.interpolate as sint
import scipy.ndimage as sim
import itertools as it

In [2]:
df = pd.read_csv('../computed_solutions_new/run_data.csv')
df.columns

Index(['Time-date', 'time', 'T_cmb0', 'deltaT0', 'r_i', 'X_Mg_0', 'X_Si_0',
       'X_O_0', 'fraction_MgFe_b', 'X_MgFeO_b', 'X_SiO2_b', 'nu_present',
       'layer_thickness', 'overturn'],
      dtype='object')

In [3]:
# Choose which columns to compute variations over
cols_to_ind = ['T_cmb0', 'deltaT0', 'X_Mg_0', 'X_Si_0',
       'X_O_0', 'fraction_MgFe_b', 'X_MgFeO_b', 'X_SiO2_b', 'nu_present',
       'layer_thickness', 'overturn']

# convert dataframe to multiindex
ri_pd = df.set_index(cols_to_ind)

# drop any column which only has one value in the dataset
for i,l in enumerate(ri_pd.index.levels):
    if len(l) ==1:
        ri_pd.index = ri_pd.index.droplevel(l.name)

# drop the time and date column
ri_pd.drop(['time','Time-date'], axis=1, inplace=True)

# convert the pandas dataframe into a numpy nd-array
shape = list(map(len,ri_pd.index.levels))
ri_arr = np.full(shape, np.nan)
ri_arr[ri_pd.index.labels] = ri_pd.values.flat

# Interpolate 

In [4]:
Nfit = 15
fcoords = [] # interpolated index locations of values in dataset
fvals = [] # values in dataset 
fnames = [] # names of fit axes
for ind in ri_pd.index.levels:
    vals = ind.values
    fnames.append(ind.name)
    mi = vals.min()
    ma = vals.max()
    fitvals = np.linspace(mi,ma,Nfit)
    fitcoords = np.interp(fitvals, vals, range(len(vals)))
    fcoords.append(fitcoords)
    fvals.append(fitvals)
    
# warning, if prodsize is > 10^7 it might take a LONG time
print('{}, prodsize={:.2e}'.format(len(fcoords), Nfit**len(fcoords)))
prod = it.product(*fcoords, repeat=1)

7, prodsize=1.71e+08


In [ ]:
# compute the inner core radius by interpolating between data
fit_ri = []
for p in prod:
    fit_ri.append(sim.map_coordinates(ri_arr, np.array([p]).T, order=1))
fit_ri = np.array(fit_ri)
fit_ri = fit_ri.reshape([len(c) for c in fcoords])

In [ ]:
# percent misfit from real inner core radius
real_ri = 1220e3
ri_poff = (fit_ri-real_ri)/real_ri-1

In [ ]:
# names of parameters that vary
fnames

In [ ]:
def plot_2ax_setothers(name1,name2, set_vals):
    '''plot 2 vars vs each other, while keeping others constant'''
    arr2plt = np.copy(ri_poff)
    i = np.array(range(arr2plt.shape[0]))
    ii,jj = np.meshgrid(i,i)
    inds = [[]]*len(arr2plt.shape)
    #inds = []*len(arr2plt.shape)
    title = ''
    for n,v in set_vals.items():
        find = fnames.index(n)
        ci = int(np.interp(v,fvals[find],range(Nfit)))
        inds[find] = ci
        if n =='nu_present':
            title +=' {}={}'.format(n,np.log10(v*5500))
        else:
            title +=' {}={}'.format(n,v)
    inds[fnames.index(name1)] = ii
    inds[fnames.index(name2)] = jj
#     print(arr2plt.shape)
    arr2plt = arr2plt[tuple(inds)]
#     print(arr2plt.shape)
    fi1 = fnames.index(name1)
    fi2 = fnames.index(name2)
    if name1 == 'nu_present':
        x = np.log10(fvals[fi1]*5500.)
    else:
        x = fvals[fi1]
    if name2 == 'nu_present':
        y = np.log10(fvals[fi2]*5500.)
    else:
        y = fvals[fi2]
    plt.pcolormesh(x,y,arr2plt, vmin=-.5, vmax=.5, cmap=plt.cm.RdBu)
    plt.colorbar(label='Ri % misfit')
    plt.xlabel(name1)
    plt.ylabel(name2)
    plt.title(title)
    

In [ ]:
name1 = 'T_cmb0'
name2 = 'nu_present'
set_vals = {'X_Mg_0':1e-5, 'X_Si_0':1e-5, 'layer_thickness':100,'deltaT0':2800,'overturn':800}
plt.figure(figsize=(28,14))
Xvs = [1e-5, 0.01, 0.25, 0.05]
n = len(Xvs)
for i in range(n):
    for j in range(n):
        set_vals['X_Mg_0'] = Xvs[i]
        set_vals['X_Si_0'] = Xvs[j]
        plt.subplot(n,n,i*n+j+1)
        plot_2ax_setothers(name1, name2, set_vals)
plt.tight_layout()
plt.savefig('../computed_solutions_new/plots/TvsNu.png')
plt.close()

In [ ]:
name1 = 'X_Si_0'
name2 = 'X_Mg_0'
set_vals = {'T_cmb0':5200, 'nu_present':10**20.5/5500, 'layer_thickness':100,'deltaT0':2800,'overturn':800}
plt.figure(figsize=(28,16))
Tcmbs = [5000,5100,5200,5300,5400]
n = len(Tcmbs)
nus = (10**np.linspace(20,21,n))/5500
for i in range(n):
    for j in range(n):
        set_vals['T_cmb0'] = Tcmbs[i]
        set_vals['nu_present'] = nus[j]
        plt.subplot(n,n,i*n+j+1)
        plot_2ax_setothers(name1, name2, set_vals)
#         plt.colorbar(label=None)
        if i==0 and j==0:
            plt.title('Tc={},nu={}'.format(Tcmbs[i], np.log10(nus[j]*5500)))
        elif i==0:
            plt.title('nu={}'.format(np.log10(nus[j]*5500)))
        elif j==0:
            plt.title('Tc={}'.format(Tcmbs[i]))
        else:
            plt.title('')
            
        if j==0:
            plt.ylabel('X_Mg')
        else:
            plt.ylabel('')
            
        if i==n-1:
            plt.xlabel('X_Si')
        else:
            plt.xlabel('')
plt.tight_layout()
plt.savefig('../computed_solutions_new/plots/MgvsSi_h{:4.0f}m.png'.format(set_vals['layer_thickness']))
plt.close()
#############################################################

name1 = 'X_Si_0'
name2 = 'X_Mg_0'
set_vals = {'T_cmb0':5200, 'nu_present':10**20.5/5500, 'layer_thickness':100,'deltaT0':2800,'overturn':800}
plt.figure(figsize=(28,16))
Tcmbs = [5500,5600,5700,5800,5900]
n = len(Tcmbs)
nus = (10**np.linspace(20,21,n))/5500
for i in range(n):
    for j in range(n):
        set_vals['T_cmb0'] = Tcmbs[i]
        set_vals['nu_present'] = nus[j]
        plt.subplot(n,n,i*n+j+1)
        plot_2ax_setothers(name1, name2, set_vals)
#         plt.colorbar(label=None)
        if i==0 and j==0:
            plt.title('Tc={},nu={}'.format(Tcmbs[i], np.log10(nus[j]*5500)))
        elif i==0:
            plt.title('nu={}'.format(np.log10(nus[j]*5500)))
        elif j==0:
            plt.title('Tc={}'.format(Tcmbs[i]))
        else:
            plt.title('')
            
        if j==0:
            plt.ylabel('X_Mg')
        else:
            plt.ylabel('')
            
        if i==n-1:
            plt.xlabel('X_Si')
        else:
            plt.xlabel('')
plt.tight_layout()
plt.savefig('../computed_solutions_new/plots/MgvsSi2_h{:4.0f}m.png'.format(set_vals['layer_thickness']))
plt.close()

In [ ]:
def plot_2ax_sumelse(a1, a2):
    ''' plot 2 vars vs each other, with number of runs 
    within a certain % summed over other variables'''
    sax = list(range(len(ri_close.shape)))
    sax.remove(a1)
    sax.remove(a2)
    s = np.sum(ri_close, axis=tuple(sax))
    if fnames[a1] == 'nu_present':
        x = np.log10(fvals[a1]*5500.)
    else:
        x = fvals[a1]
    if fnames[a2] == 'nu_present':
        y = np.log10(fvals[a2]*5500.)
    else:
        y = fvals[a2]        
    xx,yy = np.meshgrid(x, y)
    plt.pcolormesh(xx,yy,s, cmap=plt.cm.jet)
    plt.xlabel(fnames[a1])
    plt.ylabel(fnames[a2])
    plt.colorbar(label='# of runs')
#     plt.title('Inner Core Radius within 10%')

In [ ]:
# compute the number of runs within a certain % of the real inner core size
# at various parameters
p_cutoff = 0.05
for p_cutoff in [0.01, 0.05, 0.1, 0.2, 0.5]:
    ri_close = np.zeros_like(fit_ri)
    ri_close[np.where(np.abs(ri_poff)<p_cutoff)] = 1.
    plt.figure(figsize=(18,14))
    # plt.suptitle('IC R within 10%')
    for a1 in range(5):
        for a2 in range(a1+1,5):
            plt.subplot(5,5,a1+a2*5+1)
            plot_2ax_sumelse(a1,a2)
    plt.tight_layout()
    plt.savefig('../computed_solutions_new/plots/Ri_within{:.0f}p.png'.format(p_cutoff*100))
    plt.close()